# Prerequsites

# Transaction 00 - Create Delta Table

In [0]:
dbutils.fs.rm("dbfs:/user/hive/warehouse/quickstart_schema.db/users_info",True)

Out[26]: True

In [0]:
%sql
DROP SCHEMA IF EXISTS quickstart_schema CASCADE;
CREATE SCHEMA quickstart_schema;
CREATE TABLE quickstart_schema.users_info(
  id INT,
  name STRING,
  region STRING
);
DESCRIBE FORMATTED quickstart_schema.users_info;

col_name,data_type,comment
id,int,null
name,string,null
region,string,null
,,
# Detailed Table Information,,
Catalog,spark_catalog,
Database,quickstart_schema,
Table,users_info,
Created Time,Tue Dec 10 05:14:49 UTC 2024,
Last Access,UNKNOWN,


In [0]:
%sql
SELECT * from quickstart_schema.users_info;

id,name,region


# Describe History

In [0]:
%sql



version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,2024-12-10T04:51:45.000+0000,6836536383695527,naveenpn.trainer@gmail.com,CREATE TABLE,"Map(isManaged -> true, description -> null, partitionBy -> [], properties -> {})",null,List(4314377066703067),1210-034629-cmqn1uuz,null,WriteSerializable,true,Map(),null,Databricks-Runtime/12.2.x-scala2.12


# Transaction 01 - Load Data into Delta Table

In [0]:
spark.read.csv(
    path="dbfs:/FileStore/synechron/user_dataset/users_001.csv",
    header=True,
    inferSchema=True,
).limit(3).select("id", "name", "region").write.format("delta").mode(
    "append"
).saveAsTable(
    "quickstart_schema.users_info"
)

spark.read.table("quickstart_schema.users_info").display()

id,name,region
1,Heather Gibbs,Virginia
2,Herrod Petersen,Arizona
3,Ocean Workman,Tennessee


# Transaction 02 - Insert Command

In [0]:
%sql

INSERT INTO quickstart_schema.users_info VALUES(4,'Krishna','INDIA')

num_affected_rows,num_inserted_rows
1,1


# Transaction 03 - Delete

In [0]:
%sql

DELETE FROM quickstart_schema.users_info where id = 2;

num_affected_rows
1


# No. of files

In [0]:
for f in dbutils.fs.ls("dbfs:/user/hive/warehouse/quickstart_schema.db/users_info"):
  print(f.path)

dbfs:/user/hive/warehouse/quickstart_schema.db/users_info/_delta_log/
dbfs:/user/hive/warehouse/quickstart_schema.db/users_info/part-00000-02d9ef3b-eab7-4be8-b3d8-4c02c1218863-c000.snappy.parquet
dbfs:/user/hive/warehouse/quickstart_schema.db/users_info/part-00000-796d6daf-6b5b-4ff7-acf9-2fc1e0e41ec3-c000.snappy.parquet


# Select Versions

In [0]:
%sql

SELECT * from quickstart_schema.users_info;

id,name,region
1,Heather Gibbs,Virginia
3,Ocean Workman,Tennessee
4,Krishna,INDIA


In [0]:
dbutils.fs.cp("dbfs:/user/hive/warehouse/quickstart_schema.db/users_info/part-00000-02d9ef3b-eab7-4be8-b3d8-4c02c1218863-c000.snappy.parquet","/FileStore/")

Out[35]: True

In [0]:
spark.read.parquet("/FileStore/part-00000-02d9ef3b-eab7-4be8-b3d8-4c02c1218863-c000.snappy.parquet").display()

id,name,region
4,Krishna,INDIA


In [0]:
%sql

SELECT * from quickstart_schema.users_info VERSION AS OF 3

id,name,region
1,Heather Gibbs,Virginia
3,Ocean Workman,Tennessee
4,Krishna,INDIA


In [0]:
spark.read.format("text").load(
    "dbfs:/user/hive/warehouse/quickstart_schema.db/users_info/_delta_log/00000000000000000003.json"
).display()

value
"{""commitInfo"":{""timestamp"":1733807777428,""userId"":""6836536383695527"",""userName"":""naveenpn.trainer@gmail.com"",""operation"":""DELETE"",""operationParameters"":{""predicate"":""[\""(id#11098 = 2)\""]""},""notebook"":{""notebookId"":""4314377066703067""},""clusterId"":""1210-034629-cmqn1uuz"",""readVersion"":2,""isolationLevel"":""WriteSerializable"",""isBlindAppend"":false,""operationMetrics"":{""numRemovedFiles"":""1"",""numRemovedBytes"":""1158"",""numCopiedRows"":""2"",""numDeletionVectorsAdded"":""0"",""numDeletionVectorsRemoved"":""0"",""numAddedChangeFiles"":""0"",""executionTimeMs"":""1554"",""numDeletedRows"":""1"",""scanTimeMs"":""827"",""numAddedFiles"":""1"",""numAddedBytes"":""1126"",""rewriteTimeMs"":""727""},""engineInfo"":""Databricks-Runtime/12.2.x-scala2.12"",""txnId"":""72d1265d-3c7e-471d-8b43-9c84dc405670""}}"
"{""remove"":{""path"":""part-00000-6216a0e1-5778-4fce-90b7-37d93a644b3a-c000.snappy.parquet"",""deletionTimestamp"":1733807777427,""dataChange"":true,""extendedFileMetadata"":true,""partitionValues"":{},""size"":1158,""tags"":{""INSERTION_TIME"":""1733807704000000"",""MIN_INSERTION_TIME"":""1733807704000000"",""MAX_INSERTION_TIME"":""1733807704000000"",""OPTIMIZE_TARGET_SIZE"":""268435456""}}}"
"{""add"":{""path"":""part-00000-796d6daf-6b5b-4ff7-acf9-2fc1e0e41ec3-c000.snappy.parquet"",""partitionValues"":{},""size"":1126,""modificationTime"":1733807778000,""dataChange"":true,""stats"":""{\""numRecords\"":2,\""minValues\"":{\""id\"":1,\""name\"":\""Heather Gibbs\"",\""region\"":\""Tennessee\""},\""maxValues\"":{\""id\"":3,\""name\"":\""Ocean Workman\"",\""region\"":\""Virginia\""},\""nullCount\"":{\""id\"":0,\""name\"":0,\""region\"":0}}"",""tags"":{""MAX_INSERTION_TIME"":""1733807704000000"",""INSERTION_TIME"":""1733807704000000"",""MIN_INSERTION_TIME"":""1733807704000000"",""OPTIMIZE_TARGET_SIZE"":""268435456""}}}"


# Set Retention to minutes

In [0]:
%sql

ALTER TABLE quickstart_schema.users_info
SET TBLPROPERTIES ("delta.deletedFileRetentionDuration" = 'interval 3 minutes')

In [0]:
%sql

DESCRIBE DETAIL quickstart_schema.users_info;

format,id,name,description,location,createdAt,lastModified,partitionColumns,numFiles,sizeInBytes,properties,minReaderVersion,minWriterVersion,tableFeatures,statistics
delta,faf0b69f-216f-4ad5-ae29-a0081dda07e3,spark_catalog.quickstart_schema.users_info,null,dbfs:/user/hive/warehouse/quickstart_schema.db/users_info,2024-12-10T05:14:46.731+0000,2024-12-10T05:31:56.000+0000,List(),2,2202,Map(delta.deletedFileRetentionDuration -> interval 3 minutes),1,2,"List(appendOnly, invariants)",Map()


In [0]:
%sql

SELECT * from quickstart_schema.users_info;

id,name,region
1,Heather Gibbs,Virginia
3,Ocean Workman,Tennessee
4,Krishna,INDIA


In [0]:
spark.conf.set("spark.databricks.delta.retentionDurationCheck.enabled","false")

In [0]:

spark.sql("VACUUM quickstart_schema.users_info RETAIN 0.05 HOURS DRY RUN").show(truncate=False)

+-----------------------------------------------------------------------------------------------------------------------------+
|path                                                                                                                         |
+-----------------------------------------------------------------------------------------------------------------------------+
|dbfs:/user/hive/warehouse/quickstart_schema.db/users_info/part-00000-6216a0e1-5778-4fce-90b7-37d93a644b3a-c000.snappy.parquet|
+-----------------------------------------------------------------------------------------------------------------------------+



In [0]:
%sql

VACUUM quickstart_schema.users_info RETAIN 0.05 HOURS;

path
dbfs:/user/hive/warehouse/quickstart_schema.db/users_info
